In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
##from imblearn.over_sampling import SMOTE  # Import SMOTE for oversampling
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from catboost import CatBoostClassifier

In [3]:
pip uninstall pyarrow

^C
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install cffi

1.4.4


In [5]:
dados = pd.read_csv("dados/DadosFinalPt1.csv", index_col = 0)

In [8]:
dados2 = dados.drop(['CustomerID', 'Total'],axis = 1).copy()

In [170]:
dados2.dtypes

Churn                 int64
Gender               object
SeniorCitizen        object
Partner              object
Dependents           object
Tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
Monthly             float64
Total               float64
dtype: object

In [5]:
var_cat = [colname for colname, coltype in dados2.dtypes.items() if dados2[colname].dtype == object]

In [6]:
df = pd.get_dummies(dados2[var_cat], drop_first=True, dtype = np.int64)

In [172]:
df['Tenure'] = MinMaxScaler().fit_transform(dados2['Tenure'].values.reshape(len(df),1))

df['Monthly'] = MinMaxScaler().fit_transform(dados2['Monthly'].values.reshape(len(df),1))

df['Total'] = MinMaxScaler().fit_transform(dados2['Total'].values.reshape(len(df),1))

In [8]:
df

,Gender_Male,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,...,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Tenure,Monthly,Total
0,0,0,1,1,1,0,0,0,0,0,...,0,1,0,1,0,0,1,0.125000,0.471144,0.066294
1,1,0,0,0,1,0,1,0,0,0,...,1,0,0,0,0,0,1,0.125000,0.414428,0.060420
2,1,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,1,0,0.055556,0.553731,0.030239
3,1,1,1,0,1,0,0,1,0,0,...,1,0,0,1,0,1,0,0.180556,0.793532,0.140670
4,0,1,1,0,1,0,0,1,0,0,...,0,0,0,1,0,0,1,0.041667,0.653234,0.028687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0.180556,0.367164,0.083556
7263,1,0,1,0,1,0,1,1,0,0,...,1,0,0,1,0,1,0,0.305556,0.665174,0.214043
7264,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0.027778,0.318905,0.008533
7265,1,0,1,1,1,0,0,0,0,0,...,1,0,1,0,0,0,1,0.930556,0.493532,0.531831


In [26]:
df.columns


Index(['Gender_Male', 'SeniorCitizen_Yes', 'Partner_Yes', 'Dependents_Yes',
       'PhoneService_Yes', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No internet service',
       'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No internet service', 'StreamingMovies_Yes',
       'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'Tenure', 'Monthly', 'Total'],
      dtype='object')

In [203]:
data = {'Gender': 'Female', 'SeniorCitizen': 'No', 'Partner': 'No', 'Dependents': 'No', 'Tenure': 10, 'PhoneService': 'No', 'OnlineBackup': 'No', 'StreamingMovies': 'No', 'MultipleLines': 'No', 'DeviceProtection': 'No', 'Contract': 'One year', 'InternetService': 'DSL', 'TechSupport': 'No', 'PaperlessBilling': 'No', 'OnlineSecurity': 'No', 'StreamingTV': 'No', 'PaymentMethod': 'Mailed check', 'Monthly': 1000.9}    
        

In [145]:
data

{'Gender': 'Female',
 'SeniorCitizen': 'No',
 'Partner': 'No',
 'Dependents': 'No',
 'Tenure': 0.0,
 'PhoneService': 'No',
 'OnlineBackup': 'No',
 'StreamingMovies': 'No',
 'MultipleLines': 'No',
 'DeviceProtection': 'No',
 'Contract': 'One year',
 'InternetService': 'DSL',
 'TechSupport': 'No',
 'PaperlessBilling': 'No',
 'OnlineSecurity': 'No',
 'StreamingTV': 'No',
 'PaymentMethod': 'Mailed check',
 'Monthly': 0.0}

In [204]:
list_features_input = []
for feature, value in data.items():
    if type(value) == str:
        list_features_input.append(feature + '_' + value)

In [205]:
list_features_input

['Gender_Female',
 'SeniorCitizen_No',
 'Partner_No',
 'Dependents_No',
 'PhoneService_No',
 'OnlineBackup_No',
 'StreamingMovies_No',
 'MultipleLines_No',
 'DeviceProtection_No',
 'Contract_One year',
 'InternetService_DSL',
 'TechSupport_No',
 'PaperlessBilling_No',
 'OnlineSecurity_No',
 'StreamingTV_No',
 'PaymentMethod_Mailed check']

In [206]:
columns_model_categorical = ['Gender_Male','SeniorCitizen_Yes','Partner_Yes',
                     'Dependents_Yes','PhoneService_Yes','MultipleLines_No phone service',
                     'MultipleLines_Yes','InternetService_Fiber optic','InternetService_No',
                     'OnlineSecurity_No internet service',
                     'OnlineSecurity_Yes','OnlineBackup_No internet service',
                     'OnlineBackup_Yes','DeviceProtection_No internet service',
                     'DeviceProtection_Yes','TechSupport_No internet service',
                     'TechSupport_Yes','StreamingTV_No internet service',
                     'StreamingTV_Yes','StreamingMovies_No internet service',
                     'StreamingMovies_Yes','Contract_One year','Contract_Two year',
                     'PaperlessBilling_Yes','PaymentMethod_Credit card (automatic)',
                     'PaymentMethod_Electronic check','PaymentMethod_Mailed check',
                     ]

In [207]:
data_new ={}
for i in list_features_input:
        for j in columns_model_categorical:
            if i == j:
                data_new[j] = 1
            
                

In [208]:
data_new

{'Contract_One year': 1, 'PaymentMethod_Mailed check': 1}

In [209]:
new_dict = {}
for k, v in data_new.items():
    for i in range(len(columns_model_categorical)):
        if columns_model_categorical[i] not in data_new:
            new_dict[columns_model_categorical[i]] = 0
            

In [210]:
data_new 

{'Contract_One year': 1, 'PaymentMethod_Mailed check': 1}

In [211]:
new_dict

{'Gender_Male': 0,
 'SeniorCitizen_Yes': 0,
 'Partner_Yes': 0,
 'Dependents_Yes': 0,
 'PhoneService_Yes': 0,
 'MultipleLines_No phone service': 0,
 'MultipleLines_Yes': 0,
 'InternetService_Fiber optic': 0,
 'InternetService_No': 0,
 'OnlineSecurity_No internet service': 0,
 'OnlineSecurity_Yes': 0,
 'OnlineBackup_No internet service': 0,
 'OnlineBackup_Yes': 0,
 'DeviceProtection_No internet service': 0,
 'DeviceProtection_Yes': 0,
 'TechSupport_No internet service': 0,
 'TechSupport_Yes': 0,
 'StreamingTV_No internet service': 0,
 'StreamingTV_Yes': 0,
 'StreamingMovies_No internet service': 0,
 'StreamingMovies_Yes': 0,
 'Contract_Two year': 0,
 'PaperlessBilling_Yes': 0,
 'PaymentMethod_Credit card (automatic)': 0,
 'PaymentMethod_Electronic check': 0}

In [212]:
def merge(dict1, dict2):
    res = {**dict1,**dict2}
    return res

In [213]:
all_data = merge(new_dict,data_new)

In [214]:
len(all_data)

27

In [215]:
all_data

{'Gender_Male': 0,
 'SeniorCitizen_Yes': 0,
 'Partner_Yes': 0,
 'Dependents_Yes': 0,
 'PhoneService_Yes': 0,
 'MultipleLines_No phone service': 0,
 'MultipleLines_Yes': 0,
 'InternetService_Fiber optic': 0,
 'InternetService_No': 0,
 'OnlineSecurity_No internet service': 0,
 'OnlineSecurity_Yes': 0,
 'OnlineBackup_No internet service': 0,
 'OnlineBackup_Yes': 0,
 'DeviceProtection_No internet service': 0,
 'DeviceProtection_Yes': 0,
 'TechSupport_No internet service': 0,
 'TechSupport_Yes': 0,
 'StreamingTV_No internet service': 0,
 'StreamingTV_Yes': 0,
 'StreamingMovies_No internet service': 0,
 'StreamingMovies_Yes': 0,
 'Contract_Two year': 0,
 'PaperlessBilling_Yes': 0,
 'PaymentMethod_Credit card (automatic)': 0,
 'PaymentMethod_Electronic check': 0,
 'Contract_One year': 1,
 'PaymentMethod_Mailed check': 1}

In [216]:
for k, v in data.items():
    if type(v) != str:
        all_data [k] = v

In [217]:
all_data

{'Gender_Male': 0,
 'SeniorCitizen_Yes': 0,
 'Partner_Yes': 0,
 'Dependents_Yes': 0,
 'PhoneService_Yes': 0,
 'MultipleLines_No phone service': 0,
 'MultipleLines_Yes': 0,
 'InternetService_Fiber optic': 0,
 'InternetService_No': 0,
 'OnlineSecurity_No internet service': 0,
 'OnlineSecurity_Yes': 0,
 'OnlineBackup_No internet service': 0,
 'OnlineBackup_Yes': 0,
 'DeviceProtection_No internet service': 0,
 'DeviceProtection_Yes': 0,
 'TechSupport_No internet service': 0,
 'TechSupport_Yes': 0,
 'StreamingTV_No internet service': 0,
 'StreamingTV_Yes': 0,
 'StreamingMovies_No internet service': 0,
 'StreamingMovies_Yes': 0,
 'Contract_Two year': 0,
 'PaperlessBilling_Yes': 0,
 'PaymentMethod_Credit card (automatic)': 0,
 'PaymentMethod_Electronic check': 0,
 'Contract_One year': 1,
 'PaymentMethod_Mailed check': 1,
 'Tenure': 10,
 'Monthly': 1000.9}

In [218]:
columns_scale = ['Tenure','Monthly']

In [219]:
select_columns = ['Gender_Male', 'SeniorCitizen_Yes', 'Partner_Yes', 'Dependents_Yes', 'Tenure',
       'PhoneService_Yes', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No internet service',
       'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No internet service', 'StreamingMovies_Yes',
       'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check','Monthly']

In [114]:
#tabela_final = pd.DataFrame({key:dados_completos[key] for key in select_columns}, index = [0])

In [115]:
#tabela_final

,Gender_Male,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,tenure,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Monthly
0,0,0,0,0,0.0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,1,0.0


In [220]:
final_table = pd.DataFrame(all_data,columns=select_columns, index =[0])

In [221]:
final_table

,Gender_Male,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,Tenure,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Monthly
0,0,0,0,0,10,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1000.9


In [222]:
scaler = MinMaxScaler()

In [229]:
final_table[columns_scale] = scaler.fit_transform(final_table[columns_scale])

In [224]:
final_table.head()

,Gender_Male,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,Tenure,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Monthly
0,0,0,0,0,0.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0.0


In [ ]:
np.where()

In [ ]:
X = df

In [46]:
for col in dados2.columns:
    result = dados2[col].value_counts()
    print(f"Total de Registros em {col}")
    print(result)
    print("\n")

Total de Registros em Churn
Churn
0    5174
1    1869
Name: count, dtype: int64


Total de Registros em Gender
Gender
Male      3555
Female    3488
Name: count, dtype: int64


Total de Registros em SeniorCitizen
SeniorCitizen
No     5901
Yes    1142
Name: count, dtype: int64


Total de Registros em Partner
Partner
No     3641
Yes    3402
Name: count, dtype: int64


Total de Registros em Dependents
Dependents
No     4933
Yes    2110
Name: count, dtype: int64


Total de Registros em Tenure
Tenure
1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: count, Length: 73, dtype: int64


Total de Registros em PhoneService
PhoneService
Yes    6361
No      682
Name: count, dtype: int64


Total de Registros em MultipleLines
MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64


Total de Registros em InternetService
InternetService
Fiber optic    3096
DSL            2421
No 

In [ ]:
X.shape

In [ ]:
y = dados2['Churn']

In [ ]:
X = df.drop(['Total'], axis = 1)

In [ ]:
X.head()

In [ ]:
# Load and preprocess the data (replace this with your data loading and preprocessing)
# Assuming you have a DataFrame X (features) and y (target)
# Replace this with your actual data loading and preprocessing steps
# X, y = load_and_preprocess_data()

# Split the data into training and testing sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE on the training set to handle class imbalance
#smote = SMOTE(random_state=42)
#X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Initialize the model and parameter grid
model = RandomForestClassifier()

param_grid = {
    'n_estimators': range(100,200,20),
    'criterion': ['gini','entropy'],
    'max_depth': range(2,10,2),
    'min_samples_split': range(2, 10,2) ,
    'min_samples_leaf': [2,4],
    'bootstrap': [True, False]
    
    
    
    # Add other hyperparameters to tune
}

# Initialize StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV with the model, parameter grid, and scoring metric
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1',  # Use appropriate scoring metric for your problem
    cv=skf,  # StratifiedKFold as the cross-validation strategy
    verbose=2  # Controls the verbosity; adjust as needed
)

# Perform the grid search using resampled data
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding accuracy
print("Best parameters found: ", grid_search.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search.best_score_))

# Train the final model using the best hyperparameters
best_params = grid_search.best_params_
final_model = RandomForestClassifier(**best_params)
final_model.fit(X_train, y_train)

# Evaluate the final model on the test set
y_pred = final_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
pred_prob = final_model.predict_proba(X_test)[:,1]

auc_score = roc_auc_score(y_test, pred_prob)
classification_rep = classification_report(y_test, y_pred)
print("F1 score on the test set: {:.2f}".format(f1))
print(f"AUC_ROC", {auc_score})
print(classification_rep)

In [ ]:
# Split the data into training and testing sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize the model and parameter grid
model = SVC()

param_grid = {
    'C': [0.1, 1.0, 10.0, 100.0, 1000.0],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    # Add other hyperparameters to tune
}

# Initialize StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV with the model, parameter grid, and scoring metric
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1',  # Use appropriate scoring metric for your problem
    cv=skf,  # StratifiedKFold as the cross-validation strategy
    verbose=2  # Controls the verbosity; adjust as needed
)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding accuracy
print("Best parameters found: ", grid_search.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search.best_score_))

# Train the final model using the best hyperparameters
best_params = grid_search.best_params_
final_model = SVC(**best_params, probability=True)
final_model.fit(X_train, y_train)

# Evaluate the final model on the test set
y_pred = final_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
pred_prob = final_model.predict_proba(X_test)[:,1]

auc_score = roc_auc_score(y_test, pred_prob)
classification_rep = classification_report(y_test, y_pred)
print("F1 score on the test set: {:.2f}".format(f1))
print(f"AUC_ROC", {auc_score})
print(classification_rep)

In [ ]:
X_cat = dados2[var_cat].copy()

In [ ]:
X_cat['Tenure'] = MinMaxScaler().fit_transform(dados2['Tenure'].values.reshape(len(dados2),1))
X_cat['Monthly'] = MinMaxScaler().fit_transform(dados2['Monthly'].values.reshape(len(dados2),1))
X_cat['Total'] = MinMaxScaler().fit_transform(dados2['Monthly'].values.reshape(len(dados2),1))

In [ ]:
# Split the data into training and testing sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(X_cat, y, test_size=0.2, random_state=42, stratify=y)

# Initialize the model and parameter grid
model = CatBoostClassifier()

param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5],
    # Add other hyperparameters to tune
}

# Initialize StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV with the model, parameter grid, and scoring metric
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1',  # Use appropriate scoring metric for your problem
    cv=skf,  # StratifiedKFold as the cross-validation strategy
    verbose=2  # Controls the verbosity; adjust as needed
)

# Perform the grid search
grid_search.fit(X_train, y_train, cat_features= [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

# Print the best parameters and the corresponding accuracy
print("Best parameters found: ", grid_search.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search.best_score_))

# Train the final model using the best hyperparameters
best_params = grid_search.best_params_
final_model = CatBoostClassifier(**best_params)
final_model.fit(X_train, y_train, cat_features= [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

# Evaluate the final model on the test set
y_pred = final_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
pred_prob = final_model.predict_proba(X_test)[:,1]

auc_score = roc_auc_score(y_test, pred_prob)
classification_rep = classification_report(y_test, y_pred)
print("F1 score on the test set: {:.2f}".format(f1))
print(f"AUC_ROC", {auc_score})
print(classification_rep)

## SVC SEM TOTAL

In [ ]:
# Split the data into training and testing sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize the model and parameter grid
model = SVC()

param_grid = {
    'C': [0.1, 1.0, 10.0],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    # Add other hyperparameters to tune
}

# Initialize StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV with the model, parameter grid, and scoring metric
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1',  # Use appropriate scoring metric for your problem
    cv=skf,  # StratifiedKFold as the cross-validation strategy
    verbose=2  # Controls the verbosity; adjust as needed
)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding accuracy
print("Best parameters found: ", grid_search.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search.best_score_))

# Train the final model using the best hyperparameters
best_params = grid_search.best_params_
final_model = SVC(**best_params, probability=True)
final_model.fit(X_train, y_train)

# Evaluate the final model on the test set
y_pred = final_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
pred_prob = final_model.predict_proba(X_test)[:,1]

auc_score = roc_auc_score(y_test, pred_prob)
classification_rep = classification_report(y_test, y_pred)
print("F1 score on the test set: {:.2f}".format(f1))
print(f"AUC_ROC", {auc_score})
print(classification_rep)

## sem total e sem gender

In [ ]:
# Split the data into training and testing sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize the model and parameter grid
model = SVC()

param_grid = {
    'C': [0.1, 1.0, 10.0],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    # Add other hyperparameters to tune
}

# Initialize StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV with the model, parameter grid, and scoring metric
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1',  # Use appropriate scoring metric for your problem
    cv=skf,  # StratifiedKFold as the cross-validation strategy
    verbose=2  # Controls the verbosity; adjust as needed
)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding accuracy
print("Best parameters found: ", grid_search.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search.best_score_))

# Train the final model using the best hyperparameters
best_params = grid_search.best_params_
final_model = SVC(**best_params, probability=True)
final_model.fit(X_train, y_train)

# Evaluate the final model on the test set
y_pred = final_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
pred_prob = final_model.predict_proba(X_test)[:,1]

auc_score = roc_auc_score(y_test, pred_prob)
classification_rep = classification_report(y_test, y_pred)
print("F1 score on the test set: {:.2f}".format(f1))
print(f"AUC_ROC", {auc_score})
print(classification_rep)

## sem total, gender, phone

In [ ]:
# Split the data into training and testing sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize the model and parameter grid
model = SVC()

param_grid = {
    'C': [0.1, 1.0, 10.0],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    # Add other hyperparameters to tune
}

# Initialize StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV with the model, parameter grid, and scoring metric
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1',  # Use appropriate scoring metric for your problem
    cv=skf,  # StratifiedKFold as the cross-validation strategy
    verbose=2  # Controls the verbosity; adjust as needed
)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding accuracy
print("Best parameters found: ", grid_search.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search.best_score_))

# Train the final model using the best hyperparameters
best_params = grid_search.best_params_
final_model = SVC(**best_params, probability=True)
final_model.fit(X_train, y_train)

# Evaluate the final model on the test set
y_pred = final_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
pred_prob = final_model.predict_proba(X_test)[:,1]

auc_score = roc_auc_score(y_test, pred_prob)
classification_rep = classification_report(y_test, y_pred)
print("F1 score on the test set: {:.2f}".format(f1))
print(f"AUC_ROC", {auc_score})
print(classification_rep)

In [ ]:
X

TESTE DO MAKE COLUMNS TRANSFORM

In [12]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn import set_config

set_config(transform_output="pandas")

# Colunas numéricas
numeric_features = ['Tenure', 'Monthly']
numeric_transformer = Pipeline(
    steps=[('scaler', StandardScaler())]
)

# Colunas categóricas
categorical_features = ['InternetService', 'Contract', 'PaymentMethod']
categorical_transformer = Pipeline(
    steps=[('encoder', OneHotEncoder(sparse=False))
    ]
)

# Criar o transformador ColumnTransformer usando make_column_transformer
preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
    (categorical_transformer, categorical_features),
    remainder='passthrough'
)

# Ajustar e transformar os dados
X = preprocessor.fit_transform(dados2)

# Obtém as transformações do pré-processador para as colunas categóricas
categorical_transformer = preprocessor.transformers_[1][1]

# Obtém os nomes das colunas após a codificação one-hot
encoded_feature_names = list(categorical_transformer.named_steps['encoder'].get_feature_names_out(categorical_features))


# Exibe os nomes das colunas após a transformação
print(encoded_feature_names)

['InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check']


C:\Users\t-gui\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
X

,pipeline-1__Tenure,pipeline-1__Monthly,pipeline-2__InternetService_DSL,pipeline-2__InternetService_Fiber optic,pipeline-2__InternetService_No,pipeline-2__Contract_Month-to-month,pipeline-2__Contract_One year,pipeline-2__Contract_Two year,pipeline-2__PaymentMethod_Bank transfer (automatic),pipeline-2__PaymentMethod_Credit card (automatic),...,remainder__Dependents,remainder__PhoneService,remainder__MultipleLines,remainder__OnlineSecurity,remainder__OnlineBackup,remainder__DeviceProtection,remainder__TechSupport,remainder__StreamingTV,remainder__StreamingMovies,remainder__PaperlessBilling
0,-0.951682,0.027862,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,Yes,Yes,No,No,Yes,No,Yes,Yes,No,Yes
1,-0.951682,-0.161583,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,No,Yes,Yes,No,No,No,No,No,Yes,No
2,-1.155283,0.303720,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,No,Yes,No,No,No,Yes,No,No,No,Yes
3,-0.788800,1.104706,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,No,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes
4,-1.196004,0.636080,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,No,Yes,No,No,No,No,Yes,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,-0.788800,-0.319454,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,No,Yes,No,Yes,No,No,Yes,No,No,No
7263,-0.422317,0.675963,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,No,Yes,Yes,No,No,No,No,No,Yes,Yes
7264,-1.236724,-0.480648,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,No,Yes,No,No,Yes,No,No,No,No,Yes
7265,1.410099,0.102643,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,No
